In this example, we will see GEPA evolve the whole DSPy program (not just the instruction), including modifying the structure/dataflow of the program. We will use GEPA to tune a simple dspy.ChainOfThought module for MATH questions.

In [1]:
import os

os.environ["OPENAI_API_KEY"] = input("OPENAI_API_KEY: ")

In [2]:
import dspy

In [19]:
from dspy.datasets import MATH

dataset = MATH(subset="algebra")
print(len(dataset.train), len(dataset.dev), len(dataset.test))

350 350 487


Let's inspect one example from the training set.

In [4]:
example = dataset.train[0]
print("Question:", example.question)
print("Answer:", example.answer)

Question: The doctor has told Cal O'Ree that during his ten weeks of working out at the gym, he can expect each week's weight loss to be $1\%$ of his weight at the end of the previous week. His weight at the beginning of the workouts is $244$ pounds. How many pounds does he expect to weigh at the end of the ten weeks? Express your answer to the nearest whole number.
Answer: 221


Let's define a simple DSPy program to solve this task.

Unlike dspy.GEPA that can take an instantiated DSPy module as input, here, we want to evolve the full DSPy program. Hence, a candidate here is the source code as string.

In [ ]:
program_src = """import dspy
program = dspy.Predict("question -> answer")"""

GEPA interfaces with external frameworks through an adapter. In this case, we integrate GEPA with a DspyAdapter.

In [6]:
from gepa.adapters.dspy_full_program_adapter.full_program_adapter import DspyAdapter

In [7]:
def metric_fn(example, pred, trace=None):
    score = dataset.metric(example, pred)
    if score:
        feedback_text = f"The provided answer {pred.answer} is correct."
    else:
        feedback_text = f"The provided answer {pred.answer} is incorrect. The correct answer is {example.answer}."
    return dspy.Prediction(score=score, feedback=feedback_text)

In [8]:
reflection_lm = dspy.LM(model="openai/gpt-4.1", max_tokens=32000)
adapter = DspyAdapter(
    task_lm=dspy.LM(model="openai/gpt-4.1-nano", max_tokens=32000),
    metric_fn=metric_fn,
    num_threads=32,
    reflection_lm=lambda x: reflection_lm(x)[0],
)

Let's evaluate the base program

In [17]:
adapter.evaluate(dataset.test, {"program": program_src})

2025/08/27 05:10:58 INFO dspy.evaluate.evaluate: Average Metric: 327.0 / 487 (67.1%)


The base program obtains a score of 67.1%

Let's launch the GEPA optimization. We will just use 50 training and 50 valset examples, with 500 max metric calls (allowing for less than 10 GEPA candidates to be proposed).

In [ ]:
import random

random.Random(0).shuffle(dataset.train)
random.Random(0).shuffle(dataset.dev)

In [12]:
from gepa import optimize

o = optimize(
    seed_candidate={"program": program_src},
    trainset=dataset.train[:50],
    valset=dataset.dev[:50],
    adapter=adapter,
    reflection_lm=lambda x: reflection_lm(x)[0],
    max_metric_calls=2000,
    display_progress_bar=True,
)

GEPA Optimization:   0%|                                                                                                              | 0/500 [00:00<?, ?rollouts/s]2025/08/27 05:07:43 INFO dspy.evaluate.evaluate: Average Metric: 34.0 / 50 (68.0%)


Iteration 0: Base program full valset score: 0.68
Iteration 1: Selected program 0 score: 0.68
Average Metric: 2.00 / 3 (66.7%): 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 678.47it/s]

2025/08/27 05:07:43 INFO dspy.evaluate.evaluate: Average Metric: 2.0 / 3 (66.7%)
2025/08/27 05:07:43 INFO dspy.evaluate.evaluate: Average Metric: 1.0 / 3 (33.3%)
GEPA Optimization:  11%|███████████▏                                                                                        | 56/500 [00:00<00:00, 555.61rollouts/s]


Iteration 1: Proposed new text for program: import dspy

class MathQA(dspy.Signature):
    """
    Solve the given math question step by step, showing all necessary reasoning and calculations.
    Then, provide the final answer clearly and concisely.
    """
    question = dspy.InputField(desc="A math problem to solve.")
    reasoning = dspy.OutputField(desc="Step-by-step logical reasoning and calculations leading to the answer.")
    answer = dspy.OutputField(desc="The final answer, clearly stated.")

program = dspy.ChainOfThought(MathQA)
Iteration 1: New subsample score is not better, skipping
Iteration 2: Selected program 0 score: 0.68
Average Metric: 2.00 / 3 (66.7%): 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 646.54it/s]

2025/08/27 05:07:43 INFO dspy.evaluate.evaluate: Average Metric: 2.0 / 3 (66.7%)
2025/08/27 05:07:43 INFO dspy.evaluate.evaluate: Average Metric: 1.0 / 3 (33.3%)


...


Iteration 61: All subsample scores perfect. Skipping.
Iteration 61: Reflective mutation did not propose a new candidate
Iteration 62: Selected program 4 score: 0.92
Average Metric: 2.00 / 3 (66.7%): 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 121.96it/s]

2025/08/27 05:08:10 INFO dspy.evaluate.evaluate: Average Metric: 2.0 / 3 (66.7%)



Iteration 62: Proposed new text for program: from typing import Optional
import dspy

class MathWordProblemSignature(dspy.Signature):
    """
    Solve the given math word problem step by step, showing clear, detailed reasoning in the reasoning field. In the answer field, provide ONLY the final answer, in the simplest form, with no explanation, no units, and no restatement of the question. If the answer is a fraction, use LaTeX format (e.g., \\frac{a}{b}); if an interval, use standard interval notation (e.g., [0,1)); if a probability, use the simplest exact form.
    """
    question = dspy.InputField(desc="A math word problem to solve.")
    reasoning = dspy.OutputField(desc="Step-by-step solution with equations, logical steps, and explanations. Show all work.")
    answer: str = dspy.OutputField(desc="Final answer ONLY, in the simplest form, with no explanation or restatement. Use LaTeX for fractions, interval notation for intervals, and exact values for probabilities.")

class Math

2025/08/27 05:08:10 INFO dspy.evaluate.evaluate: Average Metric: 2.0 / 3 (66.7%)
GEPA Optimization: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████▊| 499/500 [00:26<00:00, 18.68rollouts/s]

Iteration 62: New subsample score is not better, skipping


Let's see the DSPy program found by GEPA

In [21]:
print(o.best_candidate["program"])

from typing import Optional
import dspy

class MathWordProblemSignature(dspy.Signature):
    """
    Solve the given math word problem step by step, showing clear, detailed reasoning in the reasoning field. In the answer field, provide ONLY the final answer, in the simplest form, with no explanation, no units, and no restatement of the question. If the answer is a fraction, use LaTeX format (e.g., \\frac{a}{b}); if an interval, use standard interval notation (e.g., [0,1)); if a probability, use the simplest exact form.
    """
    question = dspy.InputField(desc="A math word problem to solve.")
    reasoning = dspy.OutputField(desc="Step-by-step solution with equations, logical steps, and explanations. Show all work.")
    answer: str = dspy.OutputField(desc="Final answer ONLY, in the simplest form, with no explanation or restatement. Use LaTeX for fractions, interval notation for intervals, and exact values for probabilities.")

class MathWordProblemFinalAnswer(dspy.Signature):
    ""

Evaluating the optimized program

In [18]:
adapter.evaluate(dataset.test, o.best_candidate)

2025/08/27 05:12:51 INFO dspy.evaluate.evaluate: Average Metric: 448.0 / 487 (92.0%)


We see it going from 67.1% to 92% in just a few rounds of optimization!